In [4]:
!pwd

/Users/venkateshtadinada/Documents/VS-Code-Projects/naga/scratch


In [ ]:
import json 
import pprint 
pp = pprint.PrettyPrinter(indent=4)  
import glob 

In [49]:
files = glob.glob("jsonsfiles/*.json")  
i = 0
for file in files[:]: 
    i += 1
    print(i, file) 
    f = open(file) 
    try:
        mj = json.load(f) 
        parse_json(mj,file)
    except:
        print("===========>error in ", file)

found job in  jsonsfiles/Job-SAP-R3-COPY.json jobs/Job-SAP-R3-COPY.json
found job in  jsonsfiles/Job-Database-SQLScript-1.json jobs/Job-Database-SQLScript.json
found job in  jsonsfiles/Job-Hadoop-HDFSCommands.json jobs/Job-Hadoop-HDFSCommands.json
found job in  jsonsfiles/Job-Hadoop-Spark-Python-1.json jobs/Job-Hadoop-Spark-Python.json
found job in  jsonsfiles/Job-Command.json jobs/Job-Command.json
found job in  jsonsfiles/Job-FileTransfer-4.json jobs/Job-FileTransfer.json
found job in  jsonsfiles/Job-Azure-LogicApps.json jobs/Job-Azure-LogicApps.json
found job in  jsonsfiles/Job-SLAManagement.json jobs/Job-Command.json
found job in  jsonsfiles/Job-SLAManagement.json jobs/Job-SLAManagement.json
found job in  jsonsfiles/Job-Hadoop-MapReduce.json jobs/Job-Hadoop-MapReduce.json
found job in  jsonsfiles/Job-Hadoop-Tajo-Query.json jobs/Job-Hadoop-Tajo-Query.json
found job in  jsonsfiles/Job-Hadoop-Sqoop.json jobs/Job-Hadoop-Sqoop.json
found job in  jsonsfiles/Job-Azure-BatchAccount.json job

In [50]:
def parse_json(j,f):
    if type(j) is not dict:
        print ("===============>",f, " does not have a job")
        return
    for k,v in j.items():
        if type(v) is dict:
            parse_json(v,f)
    if 'Type' in j:
        if j["Type"][:3] == "Job":            
            fname = "jobs/"+j["Type"].replace(":","-")+".json"
            print ("      found job in ",f,fname) 
            with open(fname, 'w') as out_file:
                out_file.write(json.dumps(j,indent=4))
files = glob.glob("jsonsfiles/*.json")  
i = 0
for file in files[:]: 
    i += 1
    print(i, file) 
    f = open(file) 
    mj = json.load(f) 
    parse_json(mj,file)

1 jsonsfiles/Job-SAP-R3-COPY.json
      found job in  jsonsfiles/Job-SAP-R3-COPY.json jobs/Job-SAP-R3-COPY.json
2 jsonsfiles/Job-Database-SQLScript-1.json
      found job in  jsonsfiles/Job-Database-SQLScript-1.json jobs/Job-Database-SQLScript.json
3 jsonsfiles/Job-Hadoop-HDFSCommands.json
      found job in  jsonsfiles/Job-Hadoop-HDFSCommands.json jobs/Job-Hadoop-HDFSCommands.json
4 jsonsfiles/Job-Hadoop-Spark-Python-1.json
      found job in  jsonsfiles/Job-Hadoop-Spark-Python-1.json jobs/Job-Hadoop-Spark-Python.json
5 jsonsfiles/Job-Command.json
      found job in  jsonsfiles/Job-Command.json jobs/Job-Command.json
6 jsonsfiles/Job-FileTransfer-4.json
      found job in  jsonsfiles/Job-FileTransfer-4.json jobs/Job-FileTransfer.json
7 jsonsfiles/Job-Azure-LogicApps.json
      found job in  jsonsfiles/Job-Azure-LogicApps.json jobs/Job-Azure-LogicApps.json
8 jsonsfiles/Job-SLAManagement.json
      found job in  jsonsfiles/Job-SLAManagement.json jobs/Job-Command.json
      found job in  

In [114]:
def gen_name(n):
    import re
    n = re.sub('([A-Z]{1})', r'_\1',n).lower()
    if n[0] == "_":
        n = n[1:]
    return n

def get_job_name(s):
    s =  s.split(":")[-1]
    return s
def json_to_job(j):
    job_type = j["Type"]
    job_name = get_job_name(job_type) + "Job"
    job_name = job_name.replace(" ","_")
    folder_name = job_type.replace(":","/").split("/")[:-1]
    folder_name = "../new" + "/".join(folder_name) 
    folder_name = folder_name.replace("Job","Jobs")
    file_name = folder_name + "/" + job_name + ".py"
    print(job_type, file_name, job_name)
    try:
        os.makedirs(folder_name)
        with open(folder_name+"/__init__.py","w") as f:
            f.write("")

    except FileExistsError:
        # directory already exists
        pass
    with open(file_name,"w") as f:
        params = {}
        f.write("from naga.jobs.base import BaseJob\n")
        f.write("\n")
        f.write("class {}(BaseJob):\n".format(job_name))
        f.write("\tdef __init__(self, folder, job_name, \n")
        for k in j.keys():
            if k not in ["Type","RunAs","Host"]:
                params[k] = gen_name(k)
                 f.write("\t\t\tif {} not None:},\n".format(params[k]))             
                f.write("\t\t\t\t{},\n".format(params[k]))
        f.write("\t\t\thost=None, run_as=None, description=None):\n")
        f.write("\t\tBaseJob.__init__(self, folder, job_name, description=description, host=host, run_as=run_as)\n")
        for k in params:
            f.write("\t\tself.{} = {}\n".format(params[k],params[k]))      
        
        f.write("\n")
        f.write("\tdef get_json(self):\n")
        f.write("\t\tjob_json = BaseJob.get_json(self)\n")
        f.write("\t\tjob_json['Type'] = '{}'\n".format(job_type))
        for k in params:
            f.write("\t\tjob_json['{}'] = self.{}\n".format(k,params[k]) )
        f.write("\t\treturn job_json\n")

 






files = glob.glob("jobs/*.json")  
for file in files[:]: 
    f = open(file) 
    mj = json.load(f) 
    json_to_job(mj)

Job:SAP:R3:COPY ../newJobs/SAP/R3/COPYJob.py COPYJob
Job:Hadoop:HDFSCommands ../newJobs/Hadoop/HDFSCommandsJob.py HDFSCommandsJob
Job:Command ../newJobs/CommandJob.py CommandJob
Job:Azure:LogicApps ../newJobs/Azure/LogicAppsJob.py LogicAppsJob
Job:SLAManagement ../newJobs/SLAManagementJob.py SLAManagementJob
Job:Hadoop:MapReduce ../newJobs/Hadoop/MapReduceJob.py MapReduceJob
Job:Hadoop:Tajo:Query ../newJobs/Hadoop/Tajo/QueryJob.py QueryJob
Job:Hadoop:Sqoop ../newJobs/Hadoop/SqoopJob.py SqoopJob
Job:Azure:BatchAccount ../newJobs/Azure/BatchAccountJob.py BatchAccountJob
Job:PeopleSoft ../newJobs/PeopleSoftJob.py PeopleSoftJob
Job:Hadoop:Spark:ScalaJava ../newJobs/Hadoop/Spark/ScalaJavaJob.py ScalaJavaJob
Job:ADF ../newJobs/ADFJob.py ADFJob
Job:Database:MSSQL:AgentJob ../newJobs/Database/MSSQL/AgentJobJob.py AgentJobJob
Job:EmbeddedScript ../newJobs/EmbeddedScriptJob.py EmbeddedScriptJob
Job:FileWatcher:Delete ../newJobs/FileWatcher/DeleteJob.py DeleteJob
Job:Database:EmbeddedQuery ../new